In [1]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yaara.shriki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
df_niki = pd.read_json(path_or_buf="data/experiment_niki.jsonl", lines=True)
df_niki

,sentence,question,question_type,length,gender_match,correct_answer
0,The tourist hired the guides that the locals a...,Did the locals anger the guides?,filler-Q,short,mismatch,true
1,The tourists hired the guides that the locals ...,Did the locals anger the guides?,distractor-Q,short,match,true
2,The tourist hired the guides that two days ago...,Did the locals anger the guides?,filler-Q,long,mismatch,true
3,The tourists hired the guides who two days ago...,Did the locals anger the guides?,distractor-Q,long,match,true
4,The tourist hired the guides that the locals a...,Did the locals anger the tourist?,filler-Q,short,mismatch,false
...,...,...,...,...,...,...
251,The donors bored the researchers that last yea...,Did the interns disturb the researchers?,distractor-Q,long,match,true
252,The donor bored the researchers that interns d...,Did the interns disturb the donor?,filler-Q,short,mismatch,false
253,The donors bored the researchers that interns ...,Did the interns disturb the donors?,distractor-Q,short,match,false
254,The donor bored the researchers that last year...,Did the interns disturb the donor?,filler-Q,long,mismatch,false


In [23]:
# example
df_niki[df_niki["sentence"] == "The tourists hired the guides that the locals angered due to a misunderstanding"]

In [24]:
df_niki = df_niki.query("correct_answer == 'true'").reset_index(drop=True)

,sentence,question,question_type,length,gender_match,correct_answer
0,The tourist hired the guides that the locals a...,Did the locals anger the guides?,filler-Q,short,mismatch,true
1,The tourists hired the guides that the locals ...,Did the locals anger the guides?,distractor-Q,short,match,true
2,The tourist hired the guides that two days ago...,Did the locals anger the guides?,filler-Q,long,mismatch,true
3,The tourists hired the guides who two days ago...,Did the locals anger the guides?,distractor-Q,long,match,true
4,The coach removed the gymnasts that the fans d...,Did the fans disrespect the gymnasts?,filler-Q,short,mismatch,true
...,...,...,...,...,...,...
123,The editors censored the journalists that last...,Did the readers influence the journalists?,distractor-Q,long,match,true
124,The donor bored the researchers that interns d...,Did the interns disturb the researchers?,filler-Q,short,mismatch,true
125,The donors bored the researchers that interns ...,Did the interns disturb the researchers?,distractor-Q,short,match,true
126,The donor bored the researchers that last year...,Did the interns disturb the researchers?,filler-Q,long,mismatch,true


In [25]:
df_niki

In [26]:
# change the question where it is like "Did the musicians excited the producers?" to "Did the musicians excite the producers?"
df_niki.loc[df_niki.question.str.contains("heard"), "question"] = df_niki.question.str.replace("heard", "hear")
df_niki.loc[df_niki.question.str.contains("excited"), "question"] = df_niki.question.str.replace("excited", "excite")

In [27]:
# get the fourth word from the sentence
df_niki["verb"] = df_niki.question.str.split().str[3]
# df_niki["distractor"] = df_niki.question.str.split().str[3]

In [28]:
# remove the "Did" and "?" from the question
df_niki["question"] = df_niki["question"].str.replace("Did", "Therefore,").str.replace("?", "")

# get the last word from the question
df_niki["label"] = df_niki["question"].str.split().str[-1]

# change the last words of the question to <MASK>
df_niki["question"] = df_niki["question"].str.split().str[:-1].str.join(" ") + " [MASK]"

/var/folders/6f/bw7y_knd19g_q43g3ng4jbnr0000gn/T/ipykernel_11827/670632060.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_niki["question"] = df_niki["question"].str.replace("Did", "Therefore,").str.replace("?", "")


In [29]:
verbs =  df_niki["verb"].unique().tolist()
verbs_to_past_dict = {
'anger': 'angered',
'disrespect': 'disrespected',
'excite': 'excited',
'threaten': 'threatened',
'abuse': 'abused',
'care': 'cared',
'need': 'needed',
'ignore': 'ignored',
'pay': 'paid',
'watch': 'watched',
'help': 'helped',
'hear': 'heard',
'support': 'supported',
'dislike': 'disliked',
'mock': 'mocked',
'rebel': 'rebelled',
'gossip': 'gossiped',
'look': 'looked',
'fear': 'feared',
'harass': 'harassed',
'impress': 'impressed',
'compliment': 'complimented',
'admire': 'admired',
'escape': 'escaped',
'approach': 'approached',
'shoot': 'shot',
'trust': 'trusted',
'advise': 'advised',
'appreciate': 'appreciated',
'influence': 'influenced',
'disturb': 'disturbed'
}

In [30]:
df_niki["question"] = df_niki.apply(lambda row: row["question"].replace(row["verb"], verbs_to_past_dict[row["verb"]]), axis=1)

In [31]:
df_niki["prompt"] = df_niki["sentence"] + ". " + df_niki["question"]

In [32]:
# rename gender_match column to plural_match
df_niki.rename(columns={"gender_match": "plural_match"}, inplace=True)

In [33]:
nltk.download('averaged_perceptron_tagger')

sentences = df_niki["sentence"].tolist()

# Function to extract the first and second nouns from a sentence
def extract_nouns(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    nouns = [word for word, pos in tagged if pos.startswith('NN')]
    return nouns[:2]

# Extract nouns from each sentence
first_nouns = []
second_nouns = []

for sentence in sentences:
    nouns = extract_nouns(sentence)
    if len(nouns) > 0:
        first_nouns.append(nouns[0])
    else:
        first_nouns.append(None)
    if len(nouns) > 1:
        second_nouns.append(nouns[1])
    else:
        second_nouns.append(None)

# Create Pandas Series
first_nouns_series = pd.Series(first_nouns)
second_nouns_series = pd.Series(second_nouns)

# Add columns to DataFrame
df_niki["first_noun"] = first_nouns_series.values
df_niki["second_noun"] = second_nouns_series.values


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yaara.shriki/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [34]:
# add column with the wrong answer
df_niki["distractor"] = df_niki.apply(lambda row: row["first_noun"] if row["first_noun"] != row["correct_answer"] else row["second_noun"], axis=1)

In [35]:
df_niki.loc[df_niki.label == "ones"][["prompt","first_noun","second_noun","label","distractor"]]

,prompt,first_noun,second_noun,label,distractor
52,The attorney accompanied the defendants that t...,attorney,defendants,ones,attorney
53,The attorneys accompanied the defendants that ...,attorneys,defendants,ones,attorneys
54,The attorney accompanied the defendants that t...,attorney,defendants,ones,attorney
55,The attorneys accompanied the defendants that ...,attorneys,defendants,ones,attorneys


In [36]:
fix_dict = {
    ("first_noun", "ones"): "accused ones",
    ("first_noun", "ones"): "accused ones",
}

In [37]:
df_niki

,sentence,question,question_type,length,plural_match,correct_answer,verb,label,prompt,first_noun,second_noun,distractor
52,The attorney accompanied the defendants that t...,"Therefore, the jurors disliked the accused [MASK]",filler-Q,short,mismatch,true,dislike,ones,The attorney accompanied the defendants that t...,attorney,defendants,attorney
53,The attorneys accompanied the defendants that ...,"Therefore, the jurors disliked the accused [MASK]",distractor-Q,short,match,true,dislike,ones,The attorneys accompanied the defendants that ...,attorneys,defendants,attorneys
54,The attorney accompanied the defendants that t...,"Therefore, the jurors disliked the accused [MASK]",filler-Q,long,mismatch,true,dislike,ones,The attorney accompanied the defendants that t...,attorney,defendants,attorney
55,The attorneys accompanied the defendants that ...,"Therefore, the jurors disliked the accused [MASK]",distractor-Q,long,match,true,dislike,ones,The attorneys accompanied the defendants that ...,attorneys,defendants,attorneys
64,The tenant hosted the women that the neighbors...,"Therefore, the neighbors gossiped about the ol...",filler-Q,short,mismatch,true,gossip,women,The tenant hosted the women that the neighbors...,tenant,women,tenant
65,The tenants hosted the women that the neighbor...,"Therefore, the neighbors gossiped about the ol...",distractor-Q,short,match,true,gossip,women,The tenants hosted the women that the neighbor...,tenants,women,tenants
66,The tenant hosted the women that last week the...,"Therefore, the neighbors gossiped about the ol...",filler-Q,long,mismatch,true,gossip,women,The tenant hosted the women that last week the...,tenant,women,tenant
67,The tenants hosted the women that last week th...,"Therefore, the neighbors gossiped about the ol...",distractor-Q,long,match,true,gossip,women,The tenants hosted the women that last week th...,tenants,women,tenants
96,The referee disqualified the players that the ...,"Therefore, the spectators approached the tenni...",filler-Q,short,mismatch,true,approach,players,The referee disqualified the players that the ...,referee,players,referee
97,The referees disqualified the players that the...,"Therefore, the spectators approached the tenni...",distractor-Q,short,match,true,approach,players,The referees disqualified the players that the...,referees,players,referees


In [57]:
# # fix the labels
# df_niki.loc[df_niki.label == "ones", "label"] = "accused ones"
# df_niki.loc[df_niki.label == "women", "label"] = "old women"
# df_niki.loc[df_niki.label == "players", "label"] = "tennis players"
# df_niki.loc[df_niki.label == "managers", "label"] = "bank managers"

In [19]:
# drop rows
df_niki.drop(index = df_niki.query("label == 'ones' | label == 'women' | label == 'players' | label == 'managers'").index, inplace=True)

,sentence
52,The defense attorney accompanied the accused o...
53,The defense attorneys accompanied the accused ...
54,The defense attorney accompanied the accused o...
55,The defense attorneys accompanied the accused ...
64,The tenant hosted the old women that the neigh...
65,The tenants hosted the old women that the neig...
66,The tenant hosted the old women that last week...
67,The tenants hosted the old women that last wee...
96,The referee disqualified the tennis players th...
97,The referees disqualified the tennis players t...


In [21]:
df_niki

,sentence,question,question_type,length,plural_match,correct_answer,verb,label,prompt,first_noun,second_noun,distractor
52,The defense attorney accompanied the accused o...,"Therefore, the jurors disliked the accused [MASK]",filler-Q,short,mismatch,true,dislike,ones,The defense attorney accompanied the accused o...,defense,attorney,defense
53,The defense attorneys accompanied the accused ...,"Therefore, the jurors disliked the accused [MASK]",distractor-Q,short,match,true,dislike,ones,The defense attorneys accompanied the accused ...,defense,attorneys,defense
54,The defense attorney accompanied the accused o...,"Therefore, the jurors disliked the accused [MASK]",filler-Q,long,mismatch,true,dislike,ones,The defense attorney accompanied the accused o...,defense,attorney,defense
55,The defense attorneys accompanied the accused ...,"Therefore, the jurors disliked the accused [MASK]",distractor-Q,long,match,true,dislike,ones,The defense attorneys accompanied the accused ...,defense,attorneys,defense
64,The tenant hosted the old women that the neigh...,"Therefore, the neighbors gossiped about the ol...",filler-Q,short,mismatch,true,gossip,women,The tenant hosted the old women that the neigh...,tenant,women,tenant
65,The tenants hosted the old women that the neig...,"Therefore, the neighbors gossiped about the ol...",distractor-Q,short,match,true,gossip,women,The tenants hosted the old women that the neig...,tenants,women,tenants
66,The tenant hosted the old women that last week...,"Therefore, the neighbors gossiped about the ol...",filler-Q,long,mismatch,true,gossip,women,The tenant hosted the old women that last week...,tenant,women,tenant
67,The tenants hosted the old women that last wee...,"Therefore, the neighbors gossiped about the ol...",distractor-Q,long,match,true,gossip,women,The tenants hosted the old women that last wee...,tenants,women,tenants
96,The referee disqualified the tennis players th...,"Therefore, the spectators approached the tenni...",filler-Q,short,mismatch,true,approach,players,The referee disqualified the tennis players th...,referee,tennis,referee
97,The referees disqualified the tennis players t...,"Therefore, the spectators approached the tenni...",distractor-Q,short,match,true,approach,players,The referees disqualified the tennis players t...,referees,tennis,referees


In [60]:
df_niki.query("label == 'ones' | label == 'women' | label == 'players' | label == 'managers'").index

In [61]:
# drop all rows where the label is one of the following: "ones", "women", "players", "managers"


,sentence,question,question_type,length,plural_match,correct_answer,verb,label,prompt,first_noun,second_noun,distractor
0,The tourist hired the guides that the locals a...,"Therefore, the locals angered the [MASK]",filler-Q,short,mismatch,true,anger,guides,The tourist hired the guides that the locals a...,tourist,guides,tourist
1,The tourists hired the guides that the locals ...,"Therefore, the locals angered the [MASK]",distractor-Q,short,match,true,anger,guides,The tourists hired the guides that the locals ...,tourists,guides,tourists
2,The tourist hired the guides that two days ago...,"Therefore, the locals angered the [MASK]",filler-Q,long,mismatch,true,anger,guides,The tourist hired the guides that two days ago...,tourist,guides,tourist
3,The tourists hired the guides who two days ago...,"Therefore, the locals angered the [MASK]",distractor-Q,long,match,true,anger,guides,The tourists hired the guides who two days ago...,tourists,guides,tourists
4,The coach removed the gymnasts that the fans d...,"Therefore, the fans disrespected the [MASK]",filler-Q,short,mismatch,true,disrespect,gymnasts,The coach removed the gymnasts that the fans d...,coach,gymnasts,coach
...,...,...,...,...,...,...,...,...,...,...,...,...
123,The editors censored the journalists that last...,"Therefore, the readers influenced the [MASK]",distractor-Q,long,match,true,influence,journalists,The editors censored the journalists that last...,editors,journalists,editors
124,The donor bored the researchers that interns d...,"Therefore, the interns disturbed the [MASK]",filler-Q,short,mismatch,true,disturb,researchers,The donor bored the researchers that interns d...,donor,researchers,donor
125,The donors bored the researchers that interns ...,"Therefore, the interns disturbed the [MASK]",distractor-Q,short,match,true,disturb,researchers,The donors bored the researchers that interns ...,donors,researchers,donors
126,The donor bored the researchers that last year...,"Therefore, the interns disturbed the [MASK]",filler-Q,long,mismatch,true,disturb,researchers,The donor bored the researchers that last year...,donor,researchers,donor


In [62]:
df_niki

In [63]:
# # fix second noun
# df_niki.loc[df_niki.second_noun == "women", "second_noun"] = "old women"
# df_niki.loc[df_niki.second_noun == "women", "second_noun"] = "old women"
#
# # fix question
# df_niki["question"] = df_niki["question"].str.replace("old ", "")

In [64]:
# # fix first noun
# df_niki.loc[(df_niki.first_noun == "defense") & (df_niki.second_noun == "attorney"), "first_noun"] = "defense attorney"
# df_niki.loc[(df_niki.first_noun == "defense") & (df_niki.second_noun == "attorneys"), "first_noun"] = "defense attorneys"
#
# # fix second noun
# df_niki.loc[df_niki.second_noun == "attorney", "second_noun"] = "accused ones"
# df_niki.loc[df_niki.second_noun == "attorneys", "second_noun"] = "accused ones"
#
# # fix question
# df_niki["question"] = df_niki["question"].str.replace("accused ", "")
#
# # fix distractor
# df_niki.loc[df_niki.distractor == "defense", "distractor"] = df_niki.loc[df_niki.distractor == "defense", "first_noun"]

In [65]:
df_niki.drop(["question_type", "correct_answer", "verb"], axis=1, inplace=True)
df_niki.drop(["sentence", "question"], axis=1, inplace=True)

In [66]:
# change the order of the columns
df_niki = df_niki[["prompt", "label", "distractor", "length", "plural_match"]]

,prompt,label,distractor,length,plural_match
0,The tourist hired the guides that the locals a...,guides,tourist,short,mismatch
1,The tourists hired the guides that the locals ...,guides,tourists,short,match
2,The tourist hired the guides that two days ago...,guides,tourist,long,mismatch
3,The tourists hired the guides who two days ago...,guides,tourists,long,match
4,The coach removed the gymnasts that the fans d...,gymnasts,coach,short,mismatch
...,...,...,...,...,...
123,The editors censored the journalists that last...,journalists,editors,long,match
124,The donor bored the researchers that interns d...,researchers,donor,short,mismatch
125,The donors bored the researchers that interns ...,researchers,donors,short,match
126,The donor bored the researchers that last year...,researchers,donor,long,mismatch


In [211]:
df_niki

,prompt,label,distractor,length,plural_match
0,The tourist hired the guides that the locals a...,guides,tourist,short,mismatch
1,The tourists hired the guides that the locals ...,guides,tourists,short,match
2,The tourist hired the guides that two days ago...,guides,tourist,long,mismatch
3,The tourists hired the guides who two days ago...,guides,tourists,long,match
4,The coach removed the gymnasts that the fans d...,gymnasts,coach,short,mismatch
...,...,...,...,...,...
123,The editors censored the journalists that last...,journalists,editors,long,match
124,The donor bored the researchers that interns d...,researchers,donor,short,mismatch
125,The donors bored the researchers that interns ...,researchers,donors,short,match
126,The donor bored the researchers that last year...,researchers,donor,long,mismatch


In [212]:
df_niki.to_csv("data_processed/niki.csv", index=False)

# Gordon

In [213]:
# object extracted -> the subject is the correct answer, otherwise remove the row
# subject extracted -> if condition question == "matrix verb" -> the subject is the correct answer, otherwise the distractor is the correct answer
# subject extracted -> if condition question == "embedded verb" -> the distractor is the correct answer, otherwise the subject is the correct answer

In [214]:
df_gordon = pd.read_json(path_or_buf="data/experiment_gordon.jsonl", lines=True)
df_gordon.head(20)

,sentence,condition_sentence,condition_distractor,condition_question,distractor_value,subject_value,question_center,question
0,The banker that the barber praised climbed the...,subject_extracted,definite_description,matrix_verb,The barber,The banker,subject,The banker climbed the mountain.
1,The banker that the barber praised climbed the...,subject_extracted,definite_description,matrix_verb,The barber,The banker,distractor,The barber climbed the mountain.
2,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,subject,The banker climbed the mountain.
3,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,matrix_verb,You,The banker,distractor,You climbed the mountain.
4,The banker that Sue praised climbed the mounta...,subject_extracted,name,matrix_verb,Sue,The banker,subject,The banker climbed the mountain.
5,The banker that Sue praised climbed the mounta...,subject_extracted,name,matrix_verb,Sue,The banker,distractor,Sue climbed the mountain.
6,The banker that the barber praised climbed the...,subject_extracted,definite_description,embedded_verb,The barber,The banker,subject,The banker praised the barber.
7,The banker that the barber praised climbed the...,subject_extracted,definite_description,embedded_verb,The barber,The banker,distractor,The barber praised the banker.
8,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,embedded_verb,You,The banker,subject,The banker praised you.
9,The banker that you praised climbed the mounta...,subject_extracted,indexical_pronoun,embedded_verb,You,The banker,distractor,You praised the banker.


In [215]:
df_gordon.subject_value.value_counts()

The banker       24
The dancer       24
The clerk        24
The clown        24
The salesman     24
The plumber      24
The lawyer       24
The coach        24
The admiral      24
The tailor       24
The editor       24
The teacher      24
The violinist    24
The aunt         24
The chef         24
The poet         24
The actor        24
The governor     24
The robber       24
The judge        24
The detective    24
The waiter       24
The architect    24
The gardener     24
Name: subject_value, dtype: int64

In [9]:
df_gordon.query("condition_sentence == 'object_extracted'")

,sentence,condition_sentence,condition_distractor,condition_question,distractor_value,subject_value,question_center,question
12,The banker that praised the barber climbed the...,object_extracted,definite_description,matrix_verb,The barber,The banker,subject,The banker climbed the mountain.
13,The banker that praised the barber climbed the...,object_extracted,definite_description,matrix_verb,The barber,The banker,distractor,The barber climbed the mountain.
14,The banker that praised you climbed the mounta...,object_extracted,indexical_pronoun,matrix_verb,You,The banker,subject,The banker climbed the mountain.
15,The banker that praised you climbed the mounta...,object_extracted,indexical_pronoun,matrix_verb,You,The banker,distractor,You climbed the mountain.
16,The banker that praised Sue climbed the mounta...,object_extracted,name,matrix_verb,Sue,The banker,subject,The banker climbed the mountain.
...,...,...,...,...,...,...,...,...
571,The gardener that envied the homeowner was ver...,object_extracted,definite_description,embedded_verb,The homeowner,The gardener,distractor,The homeowner envied the gardener.
572,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,subject,The gardener envied you.
573,The gardener that envied you was very friendly.,object_extracted,indexical_pronoun,embedded_verb,You,The gardener,distractor,You envied the gardener.
574,The gardener that envied Liz was very friendly.,object_extracted,name,embedded_verb,Liz,The gardener,subject,The gardener envied Liz.


In [10]:
df_gordon.condition_question.value_counts()

matrix_verb      288
embedded_verb    288
Name: condition_question, dtype: int64

In [11]:
df_naama = pd.read_json(path_or_buf="data/experiment_naama.jsonl", lines=True)
df_naama

,sentence,question,dependency,animacy,gender,distractor,correct_answer
0,The actress who was in the French director's s...,The actress hit the vase.,S-V,animate,נקבה,Match,true
1,The camera that was in the French director's s...,The camera hit the vase.,S-V,inanimate,נקבה,Mismatch,true
2,This is the actress who this week in the Frenc...,The actress hit the vase.,F-G,animate,נקבה,Match,true
3,This is the camera that this week in the Frenc...,The camera hit the vase.,F-G,inanimate,נקבה,Mismatch,true
4,The student who was yesterday in the class of ...,The student bored the class.,S-V,animate,נקבה,Match,true
...,...,...,...,...,...,...,...
155,This is the alarm that in the morning in the s...,The officer woke up the combat soldiers.,F-G,inanimate,נקבה,Mismatch,false
156,The witness who was this week in the hardened ...,The criminal indicted the accomplice.,S-V,animate,נקבה,Match,false
157,The evidence that was this week in the hardene...,The criminal indicted the accomplice.,S-V,inanimate,נקבה,Mismatch,false
158,This is the witness who this week in the harde...,The criminal indicted the accomplice.,F-G,animate,נקבה,Match,false


In [12]:
df_naama[df_naama.sentence=="The actress who was in the French director's studio this week hit the expensive vase at the entrance."]

,sentence,question,dependency,animacy,gender,distractor,correct_answer
0,The actress who was in the French director's s...,The actress hit the vase.,S-V,animate,נקבה,Match,true
80,The actress who was in the French director's s...,The director hit the vase.,S-V,animate,נקבה,Match,false


# Data Exploration

In [13]:
df_niki.iloc[1]


sentence          The tourists hired the guides that the locals ...
question                           Did the locals anger the guides?
question_type                                          distractor-Q
length                                                        short
gender_match                                                  match
correct_answer                                                 true
Name: 1, dtype: object